**Loading the libraries** 

In [2]:
import pandas as pd
import os
import pytesseract
from PIL import Image
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
df = pd.read_excel('Example Data-20240208T214429Z-001/Example Data/ImageLabels.xlsx', usecols=['Image Name', 'Confidence', 'Instance Count', 'Label'])



# Append the instance counts >=1 to the label, e.g: '1 Human'

# Filter based on the criteria
filtered_df = df[(df['Confidence'] >= 70)]

# Append the instance counts to the label, e.g: '1 Human'
filtered_df['Label'] = filtered_df['Instance Count'].astype(str) + ' ' + filtered_df['Label']

# Aggregate labels for each image
aggregated_labels = filtered_df.groupby('Image Name')['Label'].apply(lambda x: ', '.join(x)).reset_index()

# Display the aggregated dataframe
aggregated_labels.head()

C:\Users\Hussein\AppData\Local\Temp\ipykernel_14576\2903431629.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Label'] = filtered_df['Instance Count'].astype(str) + ' ' + filtered_df['Label']


,Image Name,Label
0,(1) 18380579401063495.JPG,1 Motor Bike
1,(1) @GreyCupFestival - 109th Grey Cup.jpeg,"2 Adult, 1 Female, 7 Person, 1 Woman, 1 Male, ..."
2,(10) 17887803224903630.jpeg,"0 Animal, 1 Horse, 0 Horseback Riding, 0 Leisu..."
3,(12) 17985809330117499.jpeg,"0 Fun, 0 Vacation, 1 Person, 0 Road Trip, 1 He..."
4,(13) 18013990822817757.jpeg,"0 Clothing, 0 Coat, 0 Jacket, 0 Vest, 0 Shirt,..."


In [44]:
# Converting to a dictionary for ease of access
image_labels_dict = aggregated_labels.set_index('Image Name')['Label'].to_dict()

# Displaying the dictionary
image_labels_dict

{'(1) 18380579401063495.JPG': '1 Motor Bike',
 '(1) @GreyCupFestival - 109th Grey Cup.jpeg': '2 Adult, 1 Female, 7 Person, 1 Woman, 1 Male, 1 Man, 1 Helmet, 0 Crowd, 0 Clothing, 1 Coat, 0 Footwear, 1 Shoe, 0 Face, 0 Head, 0 Accessories, 0 Bag, 1 Handbag, 1 Glasses',
 '(10) 17887803224903630.jpeg': '0 Animal, 1 Horse, 0 Horseback Riding, 0 Leisure Activities, 0 Mammal, 1 Person, 1 Adult, 1 Female, 1 Woman',
 '(12) 17985809330117499.jpeg': '0 Fun, 0 Vacation, 1 Person, 0 Road Trip, 1 Helmet, 0 Nature, 0 Outdoors, 0 Sky, 1 Motorcycle, 0 Transportation, 0 Vehicle',
 '(13) 18013990822817757.jpeg': '0 Clothing, 0 Coat, 0 Jacket, 0 Vest, 0 Shirt, 0 T-Shirt, 0 Document, 0 Receipt, 0 Text, 0 Id Cards, 3 Passport, 0 Symbol, 0 Face, 0 Head, 1 Person, 1 Credit Card, 1 Business Card, 0 Paper',
 '(14) 17993584322154200.jpeg': '0 Advertisement, 2 Person, 0 Photography, 0 Animal, 6 Bird, 0 Nature, 0 Outdoors, 0 Poster, 0 Clothing, 1 Glove, 0 Wildlife, 0 Footwear, 2 Shoe, 1 Adult, 1 Bride, 1 Female, 0 

In [55]:
from sklearn.feature_extraction.text import CountVectorizer

# Example user inputs
user_text_prompt = "I went snow boarding the mountains"
smart_story_description = "Rockey Mountains"

# Step 4: Tokenize the user_text_prompt, smart_story_descriptions, and the image labels.

# Combine the user inputs and the image labels into a single list for tokenization
texts_to_tokenize = [user_text_prompt, smart_story_description] + list(image_labels_dict.values())

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Tokenize the texts
tokenized_texts = vectorizer.fit_transform(texts_to_tokenize)

# Convert the tokenized texts to an array for easier viewing and further processing
tokenized_texts_array = tokenized_texts.toarray()

# Show the shape of the array to confirm the tokenization
tokenized_texts_array.shape, vectorizer.get_feature_names_out()[:10]

((33, 169),
 array(['10', '11', '12', '13', '14', 'accessories', 'activities', 'adult',
        'adventure', 'advertisement'], dtype=object))

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

# and that the first two rows correspond to user_text_prompt and smart_story_description respectively

# Extract vectors for user inputs
user_text_vector = tokenized_texts_array[0].reshape(1, -1)  # User text prompt
story_desc_vector = tokenized_texts_array[1].reshape(1, -1)  # Smart story description

# Calculate cosine similarity between user inputs and all image labels
cos_sim_user_text = cosine_similarity(user_text_vector, tokenized_texts_array[2:])  # Skipping the first two
cos_sim_story_desc = cosine_similarity(story_desc_vector, tokenized_texts_array[2:])

# The result is a list of similarity scores between the user inputs and each image label vector
# Let's assume we want to find the image labels most similar to the user_text_prompt
most_similar_index = cos_sim_user_text.argmax()  # This gets the index of the highest similarity score
most_similar_image = list(image_labels_dict.keys())[most_similar_index]  # Retrieve the corresponding image name

most_similar_image, cos_sim_user_text[0, most_similar_index]

('(2) 18040499875507660.jpeg', 0.11547005383792514)